<a href="https://colab.research.google.com/github/zach-2pir/docs/blob/main/colabs/Prompt_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WithPi Prompt Optimization

This colab assumes that you already went through [Input Generation](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Input_Generation.ipynb), and now wish to optimize your system prompt

We will walk through the same `Aesop AI` example, but you can load any contract here. Let's dig in!

This should take about **15 minutes**, even if you're unfamiliar with Colab.

## Install and initialize SDK

Connect to a regular CPU Python 3 runtime.  You won't need GPUs for this notebook.

You'll need a WITHPI_API_KEY from https://play.withpi.ai.  Add it to your notebook secrets (the key symbol) on the left.

Run the cell below to install packages and load the SDK

In [2]:
%%capture

%pip install withpi httpx pandas

import os
from google.colab import userdata
from withpi import PiClient

os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

client = PiClient()

# Load contract and Dataset

Load the `Aesop AI` example and input set from Pi Labs cookbooks, or edit below to load a different one.


In [5]:
import httpx
import pandas as pd
from google.colab import data_table
from withpi.types import Contract

resp = httpx.get("https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/contracts/aesop_ai.json")

aesop_contract = Contract.model_validate_json(resp.content)

for dimension in aesop_contract.dimensions:
  print(dimension.label)
  for sub_dimension in dimension.sub_dimensions:
    print(f"\t{sub_dimension.description}")

df = pd.read_parquet("https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/datasets/aesop_ai_input.parquet")
data_table.enable_dataframe_formatter()
df


Story Structure
	Does the story have a clear beginning, middle, and end?
	Is there a conflict introduced early in the story that drives the plot?
	Is the resolution of the conflict clear and satisfying?
Character Development
	Does the story include characters that are relatable for children?
	Do the characters demonstrate growth or change by the end of the story?
	Is the dialogue between characters natural and age-appropriate?
Narrative Engagement
	Is the story engaging and likely to hold a child's interest?
	Does the story use vivid imagery to help children visualize the scenes?
	Does the story incorporate repetitive elements that aid in comprehension and retention?
Language Appropriateness
	Is the language used in the story appropriate for children's understanding?
	Is the story an appropriate length for a children's tale (not exceeding 1000 words)?
Moral and Cultural Sensitivity
	Does the story clearly convey a specified life lesson?
	Does the story reiterate the moral lesson at the

,input
0,Write a children's story in the style of Aesop...
1,Tell a fable about a crow and a fox that illus...
2,Create a story featuring a lion and a mouse th...
3,Write a fable involving a tortoise and a hare ...
4,Tell a story about a greedy dog who loses his ...
5,Spin a tale with a squirrel and an owl teachin...
6,Compose a fable with a feuding sun and wind th...
7,Dream up a story involving a hummingbird and a...
8,Tell a saga with a rabbit and a cunning crow i...
9,Craft a fable about a young rabbit needing hel...


# Generate Responses

The input set doesn't have model responses, but we need those

## Optimize your prompt

Kick off a prompt optimization run.

In [6]:
print(aesop_contract.description)

Write a children's story in the style of Aesop's Fables teaching a life lesson specified by the user


In [ ]:
import pandas as pd
from google.colab import data_table

prompt_optimization_status = client.prompt.optimize(
    contract=aesop_contract,
    dspy_optimization_type="COPRO",
    examples=[{
        "llm_input": "Tell me something different",
        "llm_output": "The lazy dog was jumped over by the quick brown fox",
    }],
    initial_system_instruction="Write a great story around the given topic.",
    model_id="gpt-4o-mini",
    tuning_algorithm="DSPY",
)

data_generation_status = client.data.inputs.generate_seeds(
    contract_description=aesop_contract.description,
    num_inputs=10,
)

data_table.enable_dataframe_formatter()
df = pd.DataFrame({"input": data_generation_status.data})
df

## Save the set

We will come back to this in a future colab, so it's useful to capture.  Store it as a Parquet table, which you can download.

In [ ]:
from google.colab import files

filename = "aesop_ai_input.parquet"
df.to_parquet(filename)
files.download(filename)

## Next Steps

Now you can try optimizing your prompt with the [Prompt Optimization](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Prompt_Optimization.ipynb) colab to put this to work.